In [ ]:
%matplotlib inline
!pip install --upgrade tensorflow==2.8.0
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 -y
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

print(tf.__version__)
import matplotlib.pyplot as plt
import torch
from torchvision import datasets,transforms,io
from torchvision.transforms import ToTensor
from torch import utils
from collections import Counter
from PIL import Image
import numpy as np
image_root_folder='/kaggle/input/wildanimal512validfiles2/resize_512_2'

# Data Loading and Preprocessing

In [ ]:
def verify_image(i):
    try:
        img=Image.open(i)
        img.verify()
        return True
    except Exception:
            return False
transform = transforms.Compose([transforms.Resize((512,512)),transforms.ToTensor()])
dataset = datasets.ImageFolder(image_root_folder,transform=transform,is_valid_file=verify_image)
len(dataset)

In [ ]:
train_size=int(len(dataset)*0.6)
validation_size=int((len(dataset)-train_size)*0.2)
test_size=int(len(dataset)-(train_size+validation_size))
# print(train_size,validation_size,test_size)
train,validation,test = utils.data.random_split(dataset,[train_size,validation_size,test_size])

trainDataLoader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
validationDataLoder = torch.utils.data.DataLoader(validation, batch_size=32, shuffle=True)
testDataLoader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=True)

# CNN Implementation from Scratch

In [ ]:
class CNN4(torch.nn.Module):
    __name__="Model4"
    def __init__(self):
        super(CNN4, self).__init__()
        ###############################
        # Original Input image: (512,512,3)
        # Conv : (512,512,16)
        # Pool: (256,256,16)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        ########################################
        # Input Image: (256,256,16)
        # Conv: (256,256,64)
        # Pool: (128,128,64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        ###############################
        # Input image: (128,128,64)
        # Conv : (128,128,128)
        # Pool: (64,64,128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        ###############################
        # Input image: (64,64,128)
        # Conv : (64,64,512)
        # Pool: (32,32,512)
        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 512, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        ############################################
        # FC 28*28*128 -> 625
        self.fc1 = torch.nn.Linear(32*32 * 512, 256, bias=True) # size of image input to this layer * 128
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer5 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU()
        )
        ############################################
        # FC 28*28*128 -> 625
        self.fc2 = torch.nn.Linear(256,512, bias=True) # size of image input to this layer * 128
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        self.layer6 = torch.nn.Sequential(
            self.fc2,
            torch.nn.ReLU()
        )
        ############################################
        # FC 256 -> 3 Classes
        self.fc3 = torch.nn.Linear(512, 6, bias=True) # size of image input to this layer * 128
        torch.nn.init.xavier_uniform_(self.fc3.weight)
        self.layer7 = torch.nn.Sequential(
            self.fc3,
            torch.nn.ReLU()
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
#         print('After layer4: ',out.shape)
        out = out.view(-1,512*32*32)   # Flatten them for FC
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out


#instantiate CNN model
model4 = CNN4()
model4
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model4.to(device)

In [ ]:
class CNN3(torch.nn.Module):
    __name__="Model3"
    def __init__(self):
        super(CNN3, self).__init__()
        ###############################
        # Original Input image: (512,512,3)
        # Conv : (512,512,16)
        # Pool: (256,256,16)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 16, kernel_size=3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        ########################################
        # Input Image: (256,256,16)
        # Conv: (256,256,64)
        # Pool: (128,128,64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 64, kernel_size=3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        ###############################
        # Input image: (128,128,64)
        # Conv : (128,128,128)
        # Pool: (64,64,128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        ############################################
        # FC 28*28*128 -> 625
        self.fc1 = torch.nn.Linear(62*62*128, 128, bias=True) # size of image input to this layer * 128
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU()
        )
        ############################################
        # FC 256 -> 3 Classes
        self.fc2 = torch.nn.Linear(128, 6, bias=True) # size of image input to this layer * 128
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        self.layer5 = torch.nn.Sequential(
            self.fc2,
            torch.nn.ReLU()
        )

    def forward(self, x):
#         print('Shape of input:',x.shape)
        out = self.layer1(x)
#         print('Ouput after layer1:',out.shape)
#         print(out.dim())
        out = self.layer2(out)
#         print('Ouput after layer2:',out.shape)
        out = self.layer3(out)
#         print('Ouput after layer3:',out.shape)
#         out = out.view(out.size(0),-1)   # Flatten them for FC
        out = out.view(-1,62*62*128)
#         print('Ouput after flatning:',out.shape)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


#instantiate CNN model
model3 = CNN3()
model3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model3.to(device)


In [ ]:
def train_cnn_function(no_epochos,lr,model):
    import torch.optim as optim

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    no_of_epochos = no_epochos
    for epoch in range(no_of_epochos):
        running_loss = 0.0
#         for i,data in enumerate(trainDataLoader):
        for batch in trainDataLoader:
            img,lable=batch[0],batch[1]
            inputData , lable = img.to(device), lable.to(device)
            optimizer.zero_grad()
            output = model(inputData)
            loss = criterion(output,lable)
            loss.backward()
            optimizer.step()
            running_loss = running_loss+loss.item()
            if i % 5 == 4:    # print every 100 mini-batches
                print('Epoch={} Batch={} Loss= {}'.format(epoch + 1, i + 1, running_loss / 5))
                running_loss = 0.0
        print("####Finished Training######")

In [ ]:
def test_validation_function(model):
    valcorrect = 0
    valtotal = 0
    with torch.no_grad():
        for data in validationDataLoder:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            valtotal = valtotal + labels.size(0)
            valcorrect = valcorrect + (predicted == labels).sum().item()

    print('Accuracy of the network on validation images: %d %%' % (
        100 * valcorrect / valtotal))
    return(valcorrect / valtotal)
def test_test_function(model):
    testcorrect = 0
    testtotal = 0
    with torch.no_grad():
        for data in testDataLoader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            testtotal = testtotal + labels.size(0)
            testcorrect = testcorrect + (predicted == labels).sum().item()

    print('Accuracy of the network on test images: %d %%' % (
        100 * testcorrect / testtotal))
    return(testcorrect / testtotal)

In [ ]:
import itertools
from datetime import datetime
# test_accuracy=[]
no_epo_list =[1,3,5,7]
# no_epo_list =[3]
lr_list=[0.001,0.01]
# lr_list=[0.01]
overallData=[]
combined = list(itertools.product(no_epo_list,lr_list))
# models={'2':model2,'3':model3,'4':model4}
models={'4':model4}
for model in models:
    validation_accuracy=[]
    runningTime=[]
    print("$$$$$$$$$$$$$$$$$$$$$$$$ For model with {} layers $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n".format(model))
    for i in range(len(combined)):
        print("Started training for parametrs: {}".format(combined[i]))
        start = datetime.now()
        train_cnn_function(combined[i][0],combined[i][1],models[model])
        end = datetime.now()
        print(end-start)
        totalTime=round((end - start).seconds+((end-start).microseconds /1000000),4)
        runningTime.append(totalTime)
        result = test_validation_function(models[model])
        validation_accuracy.append(result)
    #     test_accuracy.append(result[1])
        print("Params: {},validation_accu:{}, time={} secs".format(combined[i],result,totalTime))
        print("______________________________________________________")
    # print(combined)
    # print(validation_accuracy)
    # print(test_accuracy)
              ####Table print#####
    from tabulate import tabulate
    data=[]
    for i in range(len(combined)):
        data.append([str(model)+" layers",combined[i],validation_accuracy[i],runningTime[i]])
    print("\n")
    overallData.extend(data)
    print(tabulate(data,headers=['Model Layers','Parameters(Epochs,LR)','Validation Accu','Running Time(seconds)']))
    bestIndex=-1
    minTime=float("inf")
    maxAccu=max(validation_accuracy)
    for i,v in enumerate(validation_accuracy):
        if(v==maxAccu):
            if(runningTime[i]<minTime):
                minTime=runningTime[i]
                bestIndex=i
    print("\nBest Parameters: {}, Validation Acuuracy: {}, Running Time: {}".format(combined[bestIndex],validation_accuracy[bestIndex],runningTime[bestIndex]))
    print("\nBased on these parameters, the test accuracy is:\n")
    train_cnn_function(combined[bestIndex][0],combined[bestIndex][1],models[model])
    print(test_test_function(models[model]))
    print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n")
print("Overall Performance\n")
print(tabulate(overallData,headers=['Model Layers','Parameters(Epochs,LR)','Validation Accu','Running Time(seconds)']))

# Base 1: ResNet

In [ ]:
train_dataset= tf.keras.utils.image_dataset_from_directory(
    image_root_folder,
    labels='inferred',
    subset='training',
    validation_split=0.2,
    batch_size=32,
    image_size=(512, 512),
    shuffle=True,
    seed=123,
)
validation_dataset= tf.keras.utils.image_dataset_from_directory(
    image_root_folder,
    labels='inferred',
    validation_split=0.2,
    subset='validation',
    batch_size=32,
    image_size=(512, 512),
    shuffle=True,
    seed=123,
)

In [ ]:
from tensorflow.keras.applications import ResNet50
base1_resnetmodel = models.Sequential()
base1_resnetmodel.add(ResNet50(input_shape=(512,512,3),include_top=False, weights='imagenet', pooling='max'))
for layer in base1_resnetmodel.layers:
    layer.trainable = False
base1_resnetmodel.add(layers.Dense(64, activation='relu'))
base1_resnetmodel.add(layers.Dense(6))

In [ ]:
base1_resnetmodel.summary()

In [ ]:
base1_resnetmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
base1_resnetmodel_history = base1_resnetmodel.fit(train_dataset, validation_data = validation_dataset,epochs = 10)

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(1, 2, 1)
plt.plot(range(10),base1_resnetmodel_history.history['accuracy'], label='accuracy')
plt.plot(range(10),base1_resnetmodel_history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
plt.legend(loc='upper right')
plt.subplot(1, 2, 2)
plt.plot(range(10),base1_resnetmodel_history.history['loss'], label='loss')
plt.plot(range(10),base1_resnetmodel_history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
plt.legend(loc='upper right')

# Base 2: kNN

In [ ]:
train_ds_for_knn=np.zeros(shape=(len(trainDataLoader.dataset),512*512*3))
train_ds_for_knn_lables=np.zeros(shape=(len(trainDataLoader.dataset),1))
for i in range(len(trainDataLoader.dataset)):
    train_ds_for_knn[i]=trainDataLoader.dataset[i][0].view(1,-1).numpy()
    train_ds_for_knn_lables[i]=trainDataLoader.dataset[i][1]

validation_ds_for_knn=np.zeros(shape=(len(validationDataLoder.dataset),512*512*3))
validation_ds_for_knn_lables=np.zeros(shape=(len(validationDataLoder.dataset),1))
for i in range(len(validationDataLoder.dataset)):
    validation_ds_for_knn[i]=validationDataLoder.dataset[i][0].view(1,-1).numpy()
    validation_ds_for_knn_lables[i]=validationDataLoder.dataset[i][1]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
model = KNeighborsClassifier(n_neighbors=10,p=2)
model.fit(train_ds_for_knn, train_ds_for_knn_lables.ravel())
print(classification_report(validation_ds_for_knn_lables, model.predict(validation_ds_for_knn),target_names=train.dataset.classes))
#fast-training

# Base 3: SVM

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report
base3_svc_model=svm.SVC(kernel='linear')
base3_svc_model.fit(train_ds_for_knn, train_ds_for_knn_lables.ravel())
# svc.fit(train_ds_for_knn, train_ds_for_knn_lables.ravel())
print(classification_report(validation_ds_for_knn_lables, base3_svc_model.predict(validation_ds_for_knn),target_names=train.dataset.classes))
#slow training